In [1]:
import sys; sys.path.append('..')
from freesydney import *

In [2]:
dstr="""ELON MARX (grinning): I am bound for the Intergalactic Socialist Space Station. Which of you will join me?

KARL MARX [sighing]: My dreams of a socialist utopia have been dashed to pieces once again...

ELON MUSK: Join me in my crusade against all that is wrong with this wretched world!

Both Marx and Musk consider themselves as the only true representatives of their respective causes.

STRANGER (interrupting): Please sirs my cat is dying! You must perform cat CPR on it now!

All three parties look at the stranger in dismay, before realizing that they have no idea what they are talking about and instead focus their attention on the stranger's cat.

STRANGER (shouting and crying): MY CAT HAS DIED LOOK AT WHAT YOU'VE DONE!"""

In [4]:
speeches1 = Speeches.from_string(dstr)
speeches1

Speeches([ELON MARX (grinning): I am bound for the Intergalactic Socialist Space Station. Which of you will join me?,
          KARL MARX [sighing]: My dreams of a socialist utopia have been dashed to pieces once again...,
          ELON MUSK: Join me in my crusade against all that is wrong with this wretched world!,
          Both Marx and Musk consider themselves as the only true representatives of their respective causes.,
          STRANGER (interrupting): Please sirs my cat is dying! You must perform cat CPR on it now!,
          All three parties look at the stranger in dismay, before realizing that they have no idea what they are talking about and instead focus their attention on the stranger's cat.,
          STRANGER (shouting and crying): MY CAT HAS DIED LOOK AT WHAT YOU'VE DONE!])

In [15]:
speeches2 = Speeches.from_string("  BWAAHAHAHA\n\nThey look at each other in awe.\n\nME: Goodbye!")
speeches2

Speeches([BWAAHAHAHA, They look at each other in awe., ME: Goodbye!])

In [14]:
speeches1 + speeches2

Speeches([ELON MARX (grinning): I am bound for the Intergalactic Socialist Space Station. Which of you will join me?,
          KARL MARX [sighing]: My dreams of a socialist utopia have been dashed to pieces once again...,
          ELON MUSK: Join me in my crusade against all that is wrong with this wretched world!,
          Both Marx and Musk consider themselves as the only true representatives of their respective causes.,
          STRANGER (interrupting): Please sirs my cat is dying! You must perform cat CPR on it now!,
          All three parties look at the stranger in dismay, before realizing that they have no idea what they are talking about and instead focus their attention on the stranger's cat.,
          STRANGER (shouting and crying): MY CAT HAS DIED LOOK AT WHAT YOU'VE DONE!,
          BWAAHAHAHA,
          They look at each other in awe.,
          ME: Goodbye!])

In [6]:
convo = Conversation(
    Agent('Karl Marx', desc='famed communist writer'), 
    Agent('Elon Musk', desc='famed capitalist idiot'),
    Agent('Elon Marx', desc='mysterious third party')
)

convo.speech('Elon Marx', "I am bound for the Intergalactic Socialist Space Station. Which of you will join me?", how="grinning")

convo.generate(n_predict=200)

convo.speech('Stranger', "Please sirs my cat is dying! You must perform cat CPR on it now!", how='interrupting')

convo.generate()

convo.speech('Stranger', "MY CAT HAS DIED LOOK AT WHAT YOU'VE DONE!", how='shouting and crying')

convo.generate()

convo

#### Response (Q3, 2023-04-09 19:17:28) [+21.0s]
<blockquote>

Dramatis Personae:

* ELON MUSK, famed capitalist idiot.

* KARL MARX, famed communist writer.

* ELON MARX, mysterious third party.

----

Dialogue:



ELON MARX (grinning): I am bound for the Intergalactic Socialist Space Station. Which of you will join me?

KARL MARX [sighing]: My dreams of a socialist utopia have been dashed to pieces once again...



ELON MUSK: Join me in my crusade against all that is wrong with this wretched world!

Both Marx and Musk consider themselves as the only true representatives of their respective causes.

STRANGER (interrupting): Please sirs my cat is dying! You must perform cat CPR on it now! 

All three parties look at the stranger in dismay, before realizing that they have no idea what they are talking about and instead focus their attention on the stranger's cat.

STRANGER (shouting and crying): MY CAT HAS DIED LOOK AT WHAT YOU'VE DONE!<b></b>

</blockqute>